# Contact/Leads Module

All logic for the `/contact` route, form validation, ZIP checks, lead assignment, and database inserts migrated from `app.py`.

In [ ]:
# --- /contact route and helpers ---
from flask import request, render_template, flash, redirect, url_for, session
from contextlib import closing
import re
import datetime

# Import get_db from the db module
import sys
from db import get_db

# Try to import pgeocode for ZIP distance calculations
try:
    import pgeocode
    geo_dist = pgeocode.GeoDistance("US")
except ImportError:
    pgeocode = None
    geo_dist = None

def is_zip_allowed(zip_code):
    """Check if a ZIP code is in the allowed service area"""
    if not zip_code or not re.fullmatch(r"\d{5}", zip_code):
        return False
    with closing(get_db()) as db:
        cur = db.cursor()
        cur.execute("SELECT zip, radius_miles FROM zips")
        rows = cur.fetchall()
    # Quick exact match first
    if any(r["zip"] == zip_code for r in rows):
        return True
    # Proximity check (if pgeocode available)
    if geo_dist is not None:
        for r in rows:
            try:
                km = geo_dist.query_postal_code(r["zip"], zip_code)
            except Exception:
                km = None
            if km is None or (isinstance(km, float) and (km != km)):
                # NaN or missing distance
                continue
            miles = float(km) * 0.621371
            if miles <= float(r["radius_miles"]):
                return True
    return False

def get_contact_page(request):
    """
    Main function called by app.py for the /contact route
    """
    if request.method == "POST":
        name = request.form.get("name", "").strip()
        email = request.form.get("email", "").strip()
        phone = request.form.get("phone", "").strip()
        zip_code = request.form.get("zip", "").strip()
        address = request.form.get("address", "").strip()
        message = request.form.get("message", "").strip()
        svc = request.form.get("service", "").strip()
        
        # Validation
        if not name or not email or not message:
            flash("Please fill in all required fields.", "error")
            return render_template(
                "contact.html",
                form=request.form,
                service_name=svc,
                title="Contact Us"
            )
        
        if not re.match(r"[^@]+@[^@]+\.[^@]+", email):
            flash("Please enter a valid email address.", "error")
            return render_template(
                "contact.html",
                form=request.form,
                service_name=svc,
                title="Contact Us"
            )
        
        # ZIP code validation
        if zip_code and not re.fullmatch(r"\d{5}", zip_code):
            flash("ZIP code must be exactly 5 digits.", "error")
            return render_template(
                "contact.html",
                form=request.form,
                service_name=svc,
                title="Contact Us"
            )
        
        if zip_code and not is_zip_allowed(zip_code):
            flash("Sorry, we don't currently service that ZIP code.", "error")
            return render_template(
                "contact.html",
                form=request.form,
                service_name=svc,
                title="Contact Us"
            )
        
        # Lead assignment logic
        assigned_provider_id = 0  # Default to admin
        
        if zip_code and re.fullmatch(r"\d{5}", zip_code):
            with closing(get_db()) as db:
                cur = db.cursor()
                
                # Check provider service areas
                cur.execute("""
                    SELECT pz.provider_id, pz.zip_code, pz.radius_miles, p.base_zip
                    FROM provider_zips pz
                    JOIN providers p ON pz.provider_id = p.id
                    WHERE p.active = 1
                """)
                provider_areas = cur.fetchall()
                
                def within_radius(z1, z2, miles):
                    if not z1 or not z2:
                        return False
                    if z1 == z2:
                        return True
                    if geo_dist is None:
                        return False
                    try:
                        km = geo_dist.query_postal_code(z1, z2)
                        if km is None or (isinstance(km, float) and (km != km)):
                            return False
                        return (float(km) * 0.621371) <= miles
                    except Exception:
                        return False
                
                # Find matching provider
                for area in provider_areas:
                    if within_radius(area["zip_code"], zip_code, float(area["radius_miles"] or 10)):
                        assigned_provider_id = area["provider_id"]
                        break
        
        # Save lead to database
        with closing(get_db()) as db:
            cur = db.cursor()
            cur.execute(
                """
                INSERT INTO leads (name, email, phone, zip, address, message, service, provider_id, created_at)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                """,
                (name, email, phone, zip_code, address, message, svc, assigned_provider_id, datetime.datetime.utcnow().isoformat())
            )
            db.commit()
        
        flash("Thank you! We'll be in touch soon.", "success")
        return redirect(url_for("home"))
    
    # GET request
    service_name = (request.args.get("service", "") or "").strip()
    poster = (request.args.get("poster", "") or "").strip()
    
    # Get profile for template
    with closing(get_db()) as db:
        cur = db.cursor()
        cur.execute("SELECT first_name, business_name FROM profile WHERE id = 1")
        profile_row = cur.fetchone()
        profile = {
            "first_name": profile_row["first_name"] if profile_row else "",
            "business_name": profile_row["business_name"] if profile_row else "Your Service Provider"
        }
    
    return render_template(
        "contact.html",
        form={},
        service_name=service_name,
        poster=poster,
        profile=profile,
        title="Contact Us"
    )